# [ISpy NanoAOD: an event display for the CMS NanoAOD format](https://indico.cern.ch/event/1566263/contributions/6736107/)

### Tom McCauley
### University of Notre Dame, USA
### PyHEP 2025, CERN

## Introduction
The CMS Experiment introduced a new lightweight (~ 1-2 kB / event) format for physics analysis, [NanoAOD](https://doi.org/10.1051/epjconf/202024506002), during Run 2 of the LHC.

Stored as `ROOT` TTrees, NanoAOD can be read directly with [ROOT](https://root.cern/) or with Python libraries such as [`uproot`](https://github.com/scikit-hep/uproot5), which provides "ROOT I/O in pure Python and NumPy". 

Current CMS event displays rely on (at-least) the larger MiniAOD data tier, which requires CMS-specific software and resources and includes information not available in NanoAOD to produce displays such as this one which was created with a [browser-based application](https://github.com/cms-outreach/ispy-webgl) using [`three.js`](https://threejs.org/), a WebGL library and API.

![img](imgs/TOP-23-009_0.png)

`ispynanoaod` is a prototype Python package that provides interactive 3D visualization of NanoAOD event content within Jupyter notebooks. Built on `uproot`, [`awkward`](https://awkward-array.org/doc/main/index.html), and [`pythreejs`](https://github.com/jupyter-widgets/pythreejs) (a Python–`three.js` bridge for Jupyter widgets), it offers lightweight, synoptic views of events directly from NanoAOD.

## NanoAOD data

Let's fetch some NanoAOD data and have a quick look at the contents. CMS has released [32 collision datasets](https://opendata.cern.ch/search?q=&f=experiment%3ACMS&f=type%3ADataset%2Bsubtype%3ASimulated&f=type%3ADataset%2Bsubtype%3ACollision&f=file_type%3Ananoaod&l=list&order=desc&p=1&s=10&sort=mostrecent) of NanoAOD from 2016 and [over 20,000 corresponding simulation datasets](https://opendata.cern.ch/search?q=&f=experiment%3ACMS&f=type%3ADataset%2Bsubtype%3ASimulated&f=type%3ADataset%2Bsubtype%3ACollision&f=file_type%3Ananoaodsim&l=list&order=desc&p=1&s=10&sort=mostrecent) as open data which can be found via the [CERN Open Data Portal](https://opendata.cern.ch/). 

Let's fetch a file from the [DoubleMuon primary dataset in NANOAOD format from RunH of 2016](https://opendata.cern.ch/record/30555):

In [1]:
import os
import subprocess

file_name = 'EEB2FE3F-7CF3-BF4A-9F70-3F89FACE698E.root'
file_url = f'http://opendata.cern.ch/eos/opendata/cms/Run2016H/DoubleMuon/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v1/2510000/{file_name}'

if not (os.path.isfile(f'{file_name}')):
    subprocess.run(['curl', '-O', f'{file_url}'])

Internally, `ispynanoaod` reads in the NanoAOD file using `uproot` into `awkward` arrays using the `DataLoader` class:

In [2]:
import ispynanoaod as ispy

data_loader = ispy.DataLoader()
events = data_loader.load_root_file(filename=file_name)

Currently only a subset of the branches are read in and supported for display:

```
 DEFAULT_BRANCHES = [
	'run', 'event', 'luminosityBlock',
        'nJet', 'Jet_pt', 'Jet_eta', 'Jet_phi',
	'MET_pt', 'MET_phi',
	'nPhoton', 'Photon_pt', 'Photon_eta', 'Photon_phi',
        'nMuon', 'Muon_pt', 'Muon_eta', 'Muon_phi', 'Muon_charge',
        'nElectron', 'Electron_pt', 'Electron_eta', 'Electron_phi',
        'Electron_charge',
        'nSV', 'SV_x', 'SV_y', 'SV_z',
        'PV_x', 'PV_y', 'PV_z',
        'nFatJet', 'FatJet_pt', 'FatJet_eta', 'FatJet_phi',
        'nIsoTrack', 'IsoTrack_pt', 'IsoTrack_eta', 'IsoTrack_phi',
        'IsoTrack_charge',
    ]
```

The full list of branches and contents of this particular NanoAOD dataset can be found [here](https://opendata.cern.ch/eos/opendata/cms/dataset-semantics/NanoAOD/30555/DoubleMuon_doc.html).

In [3]:
events

<Array [{run: 281693, event: ..., ...}, ...] type='285647 * {run: uint32, e...'>

In [4]:
events[0]

<Record {run: 281693, event: 162400163, ...} type='{run: uint32, event: uin...'>

## Visualization

The API is very simple:

In [5]:
display = ispy.EventDisplay()
display.load_file(file_name)
display.display()

Renderer(camera=PerspectiveCamera(aspect=1.6, children=(DirectionalLight(color='white', position=(-15.0, 15.0,…

/Users/mccauley/pyhep2025-ispynanoaod/pyhep/lib/python3.11/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Here's an example of a bit more of the API in which we change the colors and display dimensions:

In [9]:
display = ispy.EventDisplay(
    width=800,
    height=600,
    background='#ececec'
)

display.object_factory.set_style('jet', color='#ffaa00', opacity=0.8)
display.object_factory.set_style('muon', color='#ff4444')
display.object_factory.set_style('electron', color='#0000ff')
display.object_factory.set_style('met', color='#00ff00')

display.detector_geometry.configure_eb(
    color='#4444ff',
    opacity=0.3,
)

display.load_file(file_name)
display.display()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='white', positio…

### Features

* Orbit controls: rotate, zoom, panning
* Picking of objects
* Simplified ECAL Barrel geometry for context
* Rendering of a subset of physics objects in NanoAOD
* Event navigation
* Support for versioning evolution of NanoAOD

## To-do

* Support for more objects
* Display more of the object data (in e.g. a table view)
* Selection of event range and specific events in input file
* Improved geometry support
* Expose rendering properties such as visibility, opacity, and color to widget
* Documentation 

## Summary

`ispynanoaod` is a python demonstrator of event visualization of CMS NanoAOD in a Jupyter notebook. It offers lightweight, synoptic views of events directly from NanoAOD.

The source code is on [GitHub](https://github.com/cms-outreach/ispy-nanoaod) and is installed via `pip` (current version 0.1.2):

```
pip install ispynanoaod
```

## Links

* [PyHEP](https://indico.cern.ch/event/1566263/)
* [ispy-nanoaod GitHub](https://github.com/cms-outreach/ispy-nanoaod)
* [ispy-nanoaod PyPi](https://pypi.org/project/ispynanoaod/)
* [ispy-nanoaod Zenodo](https://zenodo.org/records/17369554)
* [DoubleMuon primary dataset in NANOAOD format from RunH of 2016](https://opendata.cern.ch/record/30555)
* DoubleMuon dataset [variable list](https://opendata.cern.ch/eos/opendata/cms/dataset-semantics/NanoAOD/30555/DoubleMuon_doc.html)
* [three.js](https://threejs.org/)
* [uproot](https://github.com/scikit-hep/uproot5)
* [awkward](https://awkward-array.org/doc/main/index.html)
* [pythreejs](https://github.com/jupyter-widgets/pythreejs)

## Acknowledgements

Thanks to Ianna Osbourne, Jim Pivarski, and Achim Geiser for useful discussions.